In [1]:

# https://www.gymlibrary.ml/environments/box2d/bipedal_walker/
# https://www.youtube.com/watch?v=2Epn__SRHns
# https://github.com/pythonlessons/Reinforcement_Learning/tree/master/BipedalWalker-v3_PPO

import gym 
import pickle
import random
import numpy as np 
from time import time

c:\Users\jason\AppData\Local\Continuum\Anaconda3\lib\site-packages\gym\envs\registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(


In [2]:
POP = 64
GENS = 8
KEEP_PERCENTAGE = 0.5

EPS = 8
MAX_EP_LEN = 2000
GOAL = 300

MUT_VAR = 0.08

MODEL_PATH = "model.bin"

ARCH = (24, 256, 64, 4)

In [3]:
rng = np.random.default_rng(314)


def generate_random(size):
    rand_arr = rng.random(size)
    rand_signs = rng.choice([-1, 1], size)
    return rand_arr * rand_signs


def relu(a):
    return np.where(a > 0, a, 0)


def sigmoid(a):
    return 1 / (1 + np.exp(-a))


# Vectorize sigmoid function
sigmoid_v = np.vectorize(sigmoid)


def time_elapsed(prev, cur, round_digit=8):
    delta_time = cur - prev
    return f"\tTime elapsed: {round(delta_time, round_digit)} seconds"


In [4]:
# env = gym.make("BipedalWalker-v3")
# obs = env.reset()

# print(obs)

In [5]:
# env.render()

In [6]:
# env.step((-1, -1, -1, -1))

In [7]:
class NN:
    def __init__(self, arch, net_to_copy=None, mut_var=MUT_VAR):
        if net_to_copy is None:
            weights = [
                generate_random((arch[i + 1], arch[i])) for i in range(len(arch) - 1)
            ]

            biases = [generate_random((arch[i],)) for i in range(1, len(arch))]

        else:
            weights = net_to_copy.weights
            biases = net_to_copy.biases

            weights = [
                weight + np.random.normal(0.0, mut_var, weight.shape)
                for weight in weights
            ]
            biases = [
                bias + np.random.normal(0.0, mut_var, bias.shape) for bias in biases
            ]

        self.weights = weights
        self.biases = biases

    def forward(self, _obs):
        weights = self.weights
        biases = self.biases

        _obs = np.array(_obs)

        hidden1 = relu(weights[0] @ _obs + biases[0])

        hidden2 = relu(weights[1] @ hidden1 + biases[1])

        output = weights[2] @ hidden2 + biases[2]

        action = sigmoid_v(output)
        return action

    def evaluate(
        self, episodes=EPS, max_ep_len=MAX_EP_LEN, render_env=False, record=False
    ):
        ep_rewards = []
        env = gym.make("BipedalWalker-v3")

        if record:
            env = gym.wrappers.Monitor(env, "recording")

        env._max_episode_steps = max_ep_len

        for i_ep in range(episodes):
            _obs = env.reset()
            done = False
            score = 0

            while not done:
                if render_env:
                    env.render()

                action = self.forward(_obs)

                _obs, reward, done, _ = env.step(action)
                score += reward

            ep_rewards.append(score)

        env.close()

        return np.array(ep_rewards).max()


In [8]:
nn = NN(ARCH)
print([weight.shape for weight in nn.weights])

[(256, 24), (64, 256), (4, 64)]


In [9]:
# env = gym.make("BipedalWalker-v3")
# obs = env.reset()


In [10]:
# nn.weights

In [11]:
# action = nn.forward(obs)
# env.step(action)
# env.render()

In [12]:
class GA:
    def __init__(
        self,
        arch=ARCH,
        pop=POP,
        mut_var=MUT_VAR,
        verbose=False,
        eps=EPS,
        max_ep_len=MAX_EP_LEN,
        goal=GOAL,
        render_env=False,
        record=False,
    ):
        self.arch = arch
        self.pop = pop
        self.mut_var = mut_var
        self.verbose = verbose
        self.eps = eps
        self.max_ep_len = max_ep_len
        self.goal = goal
        self.render_env = render_env
        self.record = record

        self.nets = [NN(arch) for _ in range(pop)]

    def get_best_fit(self):
        #TODO Add skip for first {KEEP_PERCENTAGE} of population to reduce time spent evaluating
        # fitness_arr = self.fitnesses[int(POP * (1 - KEEP_PERCENTAGE)) :]
        # sorted_nets = self.nets[int(POP * (1 - KEEP_PERCENTAGE)) :]

        # self.nets = self.nets[int(POP * (1 - KEEP_PERCENTAGE)) :]
        
        fitness_arr = []
        sorted_nets = []
        
        for i, net in enumerate(self.nets):
            prev_time = time()

            fitness = self.evaluate(net)
            if self.verbose:
                print(
                    "Network:",
                    i + 1,
                    "\tFitness:",
                    round(fitness),
                    time_elapsed(prev_time, time()),
                )

            # Insertion sort
            inserted = False
            for j in range(len(fitness_arr)):
                if fitness_arr[j] > fitness:
                    fitness_arr.insert(j, fitness)
                    sorted_nets.insert(j, net)
                    inserted = True
                    break

            if not inserted:
                fitness_arr.append(fitness)
                sorted_nets.append(net)

            if fitness == self.goal:
                break

        fitnesses = np.array(fitness_arr)

        return sorted_nets, fitnesses

    def mutate_nets(self, parents):
        new_nets = [
            NN(self.arch, random.choice(parents), self.mut_var)
            for _ in range(self.pop - len(parents))
        ]
        return new_nets

    def train(self, gens=GENS):
        # children = self.nets
        
        for i in range(gens):
            prev_time = time()

            # nets, fitnesses = self.get_best_fit(children)
            nets, fitnesses = self.get_best_fit()

            if self.verbose:
                print(
                    "=====",
                    "Generation:",
                    i + 1,
                    "\tMaximum Reward:",
                    round(fitnesses.max()),
                    "\tAverage Reward:",
                    round(fitnesses.mean()),
                    time_elapsed(prev_time, time(), 3),
                    "=====",
                    "\n",
                )

            self.best_net = nets[-1]

            if fitnesses[-1] == self.goal:
                print("Callback reached: Max score obtained.")
                break

            parents = nets[int(POP * (1 - KEEP_PERCENTAGE)) :]
            children = self.mutate_nets(parents)

            self.nets = parents + children
            # self.fitnesses = fitnesses

    def load(self, path=MODEL_PATH):
        with open(path, "rb") as f:
            self.nets = pickle.load(f)

    def save(self, path=MODEL_PATH):
        with open(path, "wb") as f:
            pickle.dump(self.nets, f)

    def evaluate(self, net):
        mean_rewards = net.evaluate(
            self.eps, self.max_ep_len, self.render_env, self.record
        )
        return mean_rewards


In [13]:
ga = GA(verbose=True)
model_path = MODEL_PATH

In [14]:
ga.train()

Network: 1 	Fitness: -108 	Time elapsed: 0.24938607 seconds
Network: 2 	Fitness: -101 	Time elapsed: 0.72896338 seconds
Network: 3 	Fitness: -114 	Time elapsed: 0.469522 seconds
Network: 4 	Fitness: -112 	Time elapsed: 0.39248633 seconds
Network: 5 	Fitness: -55 	Time elapsed: 9.08306932 seconds
Network: 6 	Fitness: -112 	Time elapsed: 8.60283041 seconds
Network: 7 	Fitness: -108 	Time elapsed: 0.60469723 seconds
Network: 8 	Fitness: -120 	Time elapsed: 0.31255078 seconds
Network: 9 	Fitness: -118 	Time elapsed: 0.34009218 seconds
Network: 10 	Fitness: -114 	Time elapsed: 0.31272507 seconds
Network: 11 	Fitness: -115 	Time elapsed: 0.71155477 seconds
Network: 12 	Fitness: -114 	Time elapsed: 0.30634499 seconds
Network: 13 	Fitness: -113 	Time elapsed: 0.38198161 seconds
Network: 14 	Fitness: -102 	Time elapsed: 0.36519933 seconds
Network: 15 	Fitness: -103 	Time elapsed: 0.47674513 seconds
Network: 16 	Fitness: -111 	Time elapsed: 0.5556407 seconds
Network: 17 	Fitness: -112 	Time elap

In [74]:
ga.save(model_path)

In [81]:
ga.load(model_path)

record = False

if record:
    ga.best_net.evaluate(1, 200, True, True) # Record only records 200 steps
else:
    ga.nets[55].evaluate(1, 10000, True)